<a href="https://colab.research.google.com/github/Vakhranev/Heritage/blob/main/%D0%A8%D0%BA%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D1%81%D0%BE%D1%87%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import re
from collections import Counter

# Функция для очистки текста от знаков препинания и приведения к нижнему регистру
def clean_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем знаки препинания, кроме дефисов внутри слов
    return re.sub(r'(?<!\w)-|[^\w\s-]', '', text)

# Функция для подсчёта биграмм в тексте
def count_bigrams(text):
    tokens = text.split()
    bigrams = zip(tokens, tokens[1:])
    bigram_freq = Counter(bigrams)
    return bigram_freq

# Загрузка значений t_score из файла
def load_t_scores(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")
        print("Попробуем открыть файл в другой кодировке...")
        with open(filename, 'r', encoding='latin1') as f:
            return json.load(f)

# Функция для проверки и корректировки биграммы с ё
def check_yo_in_bigram(bigram, t_scores):
    bigram_yo = bigram.replace('ё', 'е')  # Версия биграммы с "е"

    # Сначала проверяем исходную биграмму
    if bigram in t_scores:
        return t_scores[bigram]

    # Если исходной нет, проверяем версию с "е"
    if bigram_yo in t_scores:
        return t_scores[bigram_yo]

    # Если обе версии отсутствуют, возвращаем 0
    return 0

def generate_json(input_txt, t_score_file, output_file):
    # Загрузка данных
    with open(input_txt, 'r', encoding='utf-8') as f:
        texts = f.read().strip().split('\n\n')

    t_scores = load_t_scores(t_score_file)

    result = {}

    for idx, text in enumerate(texts):
        key = f"EC12-B1-{str(idx).zfill(4)}_tagged"  # Уникальный ключ для каждого текста

        # Очистка текста
        clean_text_content = clean_text(text)

        # Токенизация текста
        tokens = clean_text_content.split()

        # Подсчёт частоты биграмм
        bigram_freq = count_bigrams(clean_text_content)
        bigram_freq = {" ".join(bigram): freq for bigram, freq in bigram_freq.items()}

        # Формирование биграмм с t_score
        bigrams_unique = {}
        for bigram, freq in bigram_freq.items():
            t_score = check_yo_in_bigram(bigram, t_scores)  # Используем t_score с проверкой ё/е
            bigrams_unique[bigram] = {"t_score": t_score}

        # Сохранение результатов
        result[key] = {
            "text": text,
            "tokens": tokens,
            "bigrams_freq": bigram_freq,
            "bigrams_unique": bigrams_unique
        }

    # Запись в файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

# Пример использования
input_txt = "school.txt"
t_score_file = "cleaned_t_score_file.json"
output_file = "final_school.json"

generate_json(input_txt, t_score_file, output_file)

In [ ]:
def debug_file(filename, problematic_char):
    with open(filename, 'rb') as f:
        f.seek(max(0, problematic_char - 100))  # Переместиться к проблемному байту
        snippet = f.read(200)  # Прочитать 200 байт вокруг проблемы
        print(f"Проблемный участок в байтах: {snippet}")

# Укажите позицию проблемы
debug_file("cococo_bigram_t_score_normalized.json", 231735296)

Проблемный участок в байтах: b'\xd1\x82\xd0\xb8\xd0\xb5 \xd0\xb3\xd0\xbe\xd1\x80\xd0\xb5\xd1\x87\xd0\xb8": 0.006381291629472205, "\xd0\xb2\xd0\xbe\xd1\x81\xd0\xbf\xd1\x80\xd0\xbe\xd0\xb8\xd0\xb7\xd0\xb2\xd0\xbe\xd0\xb4\xd0\xb8\xd0\xbc\xd1\x8b\xd0\xbc\xd0\xb8 \xd1\x81\xd0\xb2\xd0\xbe\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd0\xb0\xd0\xbc\xd0\xb8": 0.006381291629472205, "\xd0\xb2\xd0\xbe\xd1\x81\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd\xd0\xb0\xd0\xb2\xd0\xbb\xd0\xb8\xd0\xb2\xd0\xb0\xd1\x8e\xd1\x82 \xd0\xb8\xd1\x81\xd1\x82\xd0\xb8\xd0\xbd\xd1\x83": 0.006381291629472205, "\xd0\xb2\xd0\xbe\xd0\xbe\xd0'


In [ ]:
def clean_json_lines(filename, output_filename):
    import json

    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    valid_lines = []
    for line in lines:
        try:
            json.loads(line)  # Проверяем строку на корректность
            valid_lines.append(line)
        except json.JSONDecodeError:
            continue  # Пропускаем некорректные строки

    with open(output_filename, 'w', encoding='utf-8') as f:
        f.writelines(valid_lines)

    print(f"Чистый файл сохранен в {output_filename}.")

In [ ]:
clean_json_lines("cococo_bigram_t_score_normalized.json", "cleaned_t_score_file.json")

Чистый файл сохранен в cleaned_t_score_file.json.
